In [1]:
import pandas as pd
import numpy as np
from glob import glob


In [2]:

file_lst = glob("dataset_ton/*.csv")


['pkSeqID',
 'stime',
 'flgs',
 'proto',
 'saddr',
 'sport',
 'daddr',
 'dport',
 'pkts',
 'bytes',
 'state',
 'ltime',
 'seq',
 'dur',
 'mean',
 'stddev',
 'smac',
 'dmac',
 'sum',
 'min',
 'max',
 'soui',
 'doui',
 'sco',
 'dco',
 'spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'rate',
 'srate',
 'drate',
 'attack',
 'category',
 'subcategory ']

In [3]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [4]:

n_normal = 0
n_attack = 0
req_ft=['pkSeqID','stime','pkts','bytes','ltime','seq','dur','mean','stddev','sum','min','max','spkts','dpkts','sbytes','dbytes','rate','srate','drate', 'attack']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, names=features, low_memory=False)
    df.columns=features
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['attack'] == 1]
    nr_df = df[df['attack'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]
    del df
    # print(len(r))    
    # r.add(nr_df)
    # rng = np.random.default_rng()
    # r.add(rng.choice(at_df.to_numpy(), size=max(0,int(size*0.01 - n_nr)), replace=False))

    


20


In [5]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 9543 ATTACK: 73360900


In [3]:
df = r.getDf()
df.to_csv("imbalanced_dataset.csv", index=False)

NameError: name 'r' is not defined

In [4]:
df = pd.read_csv("imbalanced_dataset.csv")

In [14]:
from imblearn.over_sampling import ADASYN
Y = df['attack'].to_numpy().astype(np.uint8)
X = df.drop('attack', axis=1).to_numpy()
df[df['attack'].isnull()].head()
print(Y[0:100])



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [19]:
from imblearn.over_sampling import ADASYN
X_r, Y_r = ADASYN().fit_resample(X, Y)
res=np.empty((X_r.shape[0], df.shape[1]))
res[:, 0:-1] = X_r
res[:, -1] = Y_r
pd.DataFrame(res, columns = df.columns).to_csv("balanced_dataset.csv", index=False)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,62008333.0,1.528100e+09,1596.0,1553365.0,1.528100e+09,980.0,10.002418,4.994978,0.004577,9.989956,4.990401,4.999555,799.0,797.0,759235.0,794130.0,159.461456,79.780716,79.580757,0.0
1,62008334.0,1.528100e+09,1598.0,1557651.0,1.528100e+09,981.0,10.002407,4.994966,0.004576,9.989932,4.990390,4.999542,799.0,799.0,803753.0,753898.0,159.661575,79.780800,79.780800,0.0
2,62008335.0,1.528100e+09,799.0,766163.0,1.528100e+09,982.0,10.002138,4.994828,0.004355,9.989656,4.990207,4.999449,799.0,0.0,766163.0,0.0,79.782944,79.782944,0.000000,0.0
3,62008336.0,1.528100e+09,797.0,746889.0,1.528100e+09,983.0,10.002233,4.994583,0.004086,9.989166,4.990215,4.998951,797.0,0.0,746889.0,0.0,79.582230,79.582230,0.000000,0.0
4,62010845.0,1.528100e+09,479.0,340071.0,1.528100e+09,3492.0,10.001781,4.988259,0.001386,9.976519,4.987583,4.988935,479.0,0.0,340071.0,0.0,47.791485,47.791485,0.000000,0.0
5,62010846.0,1.528100e+09,429.0,25740.0,1.528100e+09,3493.0,10.001780,4.988258,0.001684,9.976517,4.987583,4.988934,429.0,0.0,25740.0,0.0,42.792385,42.792385,0.000000,0.0
6,62013042.0,1.528100e+09,270.0,79920.0,1.528100e+09,5689.0,9.988903,4.975750,0.001237,9.951500,4.974513,4.976987,270.0,0.0,79920.0,0.0,26.929884,26.929884,0.000000,0.0
7,62088411.0,1.528100e+09,4.0,1732.0,1.528100e+09,81058.0,2.640741,2.640741,0.000000,2.640741,2.640741,2.640741,2.0,2.0,1600.0,132.0,1.136045,0.378682,0.378682,0.0
8,62270096.0,1.528100e+09,2525.0,2456516.0,1.528100e+09,584.0,16.522152,4.121648,1.513722,16.486591,1.499813,4.998389,1264.0,1261.0,1201565.0,1254951.0,152.764603,76.442825,76.261253,0.0
9,62270097.0,1.528100e+09,2524.0,2459462.0,1.528100e+09,585.0,16.522139,4.121534,1.513662,16.486137,1.499802,4.998376,1264.0,1260.0,1269680.0,1189782.0,152.704208,76.442886,76.200790,0.0


In [20]:
np.bincount(Y_r)

array([724488, 724161])

In [30]:
np.random.shuffle(res)


In [35]:
pd.DataFrame(res, columns = df.columns).to_csv("shuffled_balanced_dataset.csv", index=False)

In [37]:
pd.DataFrame(res).tail(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1448634,4.151217e+07,1.528096e+09,5.000000,676.000000,1.528096e+09,104610.000000,11.003812,3.657313,4.822480e-01,7.314625,3.175065,4.139560,4.000000,1.000000,616.000000,60.000000,0.363510,0.272633,0.000000,1.0
1448635,6.990167e+07,1.528101e+09,2.000000,120.000000,1.528101e+09,201531.000000,15.986249,0.000000,0.000000e+00,0.000000,0.000000,0.000000,2.000000,0.000000,120.000000,0.000000,0.062554,0.062554,0.000000,1.0
1448636,1.647896e+06,1.526962e+09,2.000000,124.008264,1.526962e+09,1723.399871,0.000654,0.000654,0.000000e+00,0.000654,0.000654,0.000654,1.000000,1.000000,62.004132,62.004132,10618.616285,0.000000,0.000000,0.0
1448637,2.115933e+06,1.528081e+09,2.000000,348.838817,1.528081e+09,256075.297007,0.169840,0.169840,0.000000e+00,0.169840,0.169840,0.169840,1.000000,1.000000,97.891021,250.947796,30.361952,0.000000,0.000000,0.0
1448638,1.926339e+07,1.528086e+09,1489.013676,89340.820542,1.528086e+09,811.368485,36.949741,4.666845,7.300056e-01,35.339016,2.757596,4.990351,1489.013676,0.000000,89340.820542,0.000000,40.359632,40.359632,0.000000,0.0
1448639,1.852383e+06,1.528089e+09,3.586843,236.901322,1.528090e+09,10489.721131,948.941516,0.000153,7.934215e-07,0.000200,0.000151,0.000153,1.793421,1.793421,112.769752,124.131570,409.879960,0.000663,0.000663,0.0
1448640,1.724269e+06,1.526964e+09,2.000000,172.000000,1.526964e+09,23338.178738,0.505023,0.005351,0.000000e+00,0.005351,0.005351,0.005351,1.199799,0.800201,103.182729,68.817271,119.745044,0.079892,0.000000,0.0
1448641,1.071272e+07,1.528083e+09,4.000000,522.000000,1.528083e+09,147923.000000,39.439926,0.016304,2.305700e-02,0.048912,0.000000,0.048912,3.000000,1.000000,462.000000,60.000000,0.076065,0.050773,0.000000,1.0
1448642,4.326104e+07,1.528097e+09,707.561947,44105.272014,1.528097e+09,18359.240365,20.961057,3.788939,7.544549e-01,18.409626,2.280271,4.175149,705.385627,2.176320,43961.634864,143.637149,31.528583,31.400348,0.112205,0.0
1448643,3.317520e+06,1.528082e+09,2.000000,426.528887,1.528082e+09,146785.016787,0.221050,0.221050,0.000000e+00,0.221050,0.221050,0.221050,1.000000,1.000000,101.960926,324.567961,4.527360,0.000000,0.000000,0.0
